# Mobile Robotics Project 
## Groupe 47

| Name      | SCIPER  |
|-----------|---------|
| Martin Daigne    | 340936  |
| Laetitia Fayad | 325743  |
| Arthur Lamour   | 300443  |
| Amandine Meunier | 344736  |


# Summary
- [Introduction](#introduction)
- [The Arena](#thearena)
- [Assumptions](#assumptions) 
- [Implementation](#implementation)
- [1. Computer Vision](#computervision)
    - [1.a ArUco Markers](#arucomarkers)
        - [1.a.i Corner detection and perspective](#cornerdetection) 
        - [1.a.ii Thymio Detection](#thymiodetection)
    - [1.b Color Detection](#colordetection)
- [2. Kalman Filter](#kalmanfilter)
    - [2.a Motion Model](#motionmodel)
    - [2.b Kalman Equations](#kalmanequations)
    - [2.c Covariance measurements](#covariancemeasurements)
        - [2.c.i Process covariance Q](#processcovarianceq) 
	    - [2.c.ii Measurements Covariance R](#measurementscovariancer)
    - [2.d Code implementation](#codeimplementation)
- [3. Kidnapping](#kidnapping)
- [4. Path Planning](#pathplanning)
    - [4.a A*](#astar)
    - [4.b Finding keypoints of the path](#keypoints)
- [5. Motion Control](#motioncontrol)
    - [5.a Setting the motors' speed](#motorspeeds)
    - [5.b Controller](#controller)
    - [5.c Local Avoidance](#localavoidance)
- [Annexe](#annexe)

<a id="introduction"></a>
# Introduction

As part of the course Basics of mobile robotics, given to master's students at EPFL by professor Mondada, we did a project using a Thymio and a camera. First, with computer vision, we map the environment where the Thymio<a href="#footnote1" id="footnote1ref">[0]</a> will move, creating a grid based on the limits of the workspace and mapping global obstacles and the goal the robot has to reach. We localize the Thymio and find the shortest path it can follow to reach the goal without encountering any global obstacles. The Thymio then follows this path and can react to local obstacles added in front of him, avoiding them and continuing towards his goal. 

In our project the robot is moving on a white A0 sheet for better contrasts and flatness with 4 aruco markers on the borders to better correct the perspective. The Thymio has an aruco markers on him for detection. The global obstacles are blue paper sheets to make arbitrary shapes and move them around easily. For the same reasons, the goal is a green paper sheet. The local obstacles are represented as 5cm circular hollow objects.

<a id="footnote0"></a> [0]:[Thymio Schematics](https://moodle.epfl.ch/pluginfile.php/2706097/mod_resource/content/1/ThymioCheatSheet.pdf)

<a id="thearena"></a>
## The Arena
- Background: a white A0 sheet
- Goal: a green shape
- Global obstacles: shapes of arbitrary color (seen by the camera)
- Local obstacles (invisible to camera): circular object of diameter $\approx 5cm$, like a paprika container
- Aukey PC-W1 Webcam
- Wireless Thymio with Aruco {id:9, size:70mm}
- Top Left ArUco: {id:0, size:70mm}
- Bottom Left ArUco: {id:1, size:70mm}
- Top Right ArUco: {id:10, size:70mm}
- Bottom Right ArUco: {id:2, size:70mm}

<img src="./plots/Imageaftercorrection.png" alt="Arena" width="600">

Note: the aruco markers placement order and orientation are very important for computer vision.

<a id="assumptions"></a>
## Assumptions

- There should always be at least one path from robot to goal, global obstacles cannot completly block the robot and the goal
- Local obstacles should be placed in a way such that during local avoidance maneuvres the robot stays inside the arena and doesn't collide with global obstacles
- Goal and global obstacles should remain fixed during a run and detectable by their color
- Kalman<a href="#footnote1" id="footnote1ref">[1]</a>:
    - System is linear
    - Errors are zero mean Gaussians, with known covariance
    - Process and Measurement noise are independant
    - Markovian, $x_t$ only depends on $x_{t-1}$

<a id="footnote1"></a> [1]: [Kalman Filter and Parameter Identification by Florian Herzog, ETHZ 2013](https://ethz.ch/content/dam/ethz/special-interest/mavt/dynamic-systems-n-control/idsc-dam/Lectures/Stochastic-Systems/Kalman_Filter.pdf)

<a id="implementation"></a>
# Implementation
Here is a flow chart of the code structure. The code is divided in 5 files. The main file following the flow chart, the Thymio class containing the thymio's information, the thymio's detection, the kalman filter and the path controller. Secondly, the camera class containing the camera warmup, image capture, arena corner detection, perspective correction, obstacle and goal detection and plotting. Thirdly, the path planning with A*, keypoints and image to grid conversion. Finally, the local avoidance, with a Braintenberg approach.
We will then go through these features:
1. Computer vision : arena corners, Thymio, obstacles and goal detections, obstacles expansion
2. Path Planning : A* Algorithm, keypoints <a href="#footnote2" id="footnote2ref">[2]</a>
3. Kalman Filter : Motion equations and covariances measurements
4. Motion control : Astolfi Controller <a href="#footnote3" id="footnote3ref">[3]</a>
5. Local Avoidance : Braitenberg <a href="#footnote4" id="footnote4ref">[4]</a>

And finally we will see how they are integrated in the main programm.
This flow chart gives a good idea of the flow of the program eventhough it got changed and there are a lot more checks and small adjustements. 

<img src="https://i.postimg.cc/yNG1T81t/image-2024-12-04-151247433.png" alt="flowchart" width="700">



<a id="footnote2"></a> [2]:[Robotic Motion Planning: A* and D* Search by Howie Choset, Carnegie Mellon University](https://www.cs.cmu.edu/~motionplanning/lecture/AppH-astar-dstar_howie.pdf)

<a id="footnote3"></a> [3]:[Astolfi Controller by Francesco Mondada, EPFL, lecture 1 page 13](https://moodle.epfl.ch/pluginfile.php/2703472/mod_resource/content/11/Slides%2001%20-%20Components%20of%20a%20Mobile%20Robot.pdf)

<a id="footnote2"></a> [4]:[Reactive Controllers Behavior-Based Control Navigation & Obstacle Avoidance by GIANNI A. DI CARO, Carnegie Mellon University Qatar](https://web2.qatar.cmu.edu/~gdicaro/16311-Fall17/slides/16311-13-Behaviors-Navigation.pdf)

<a id="computervision"></a>
## 1. Computer vision
This section will present the computer vision implementation. We used two different detection methods: The aruco markers and the color detection.

As the camera has automatic white balance (AWB) and automatic exposure (AE), the very first frames appear yellowish. To give the camera time to adapt to the scene, we take 50 images:
```python
for _ in range(50):  # Camera needs to do lighting and white balance adjustments
    ret, _ = self.cam.read()
    if not ret:
        self.cam.release()
        raise IOError("Failed to capture frame during warmup")
```
We can see the usufulness of the warm up:
<img src="./plots/Yellow.png" alt="Arena" width="600">
<img src="./plots/Imagebeforecorrection.png" alt="Arena" width="600">

By default the camera has a resolution of 640x480, it allows efficient computing and detection. It can be set to 1080p, for a larger arena but it will impact performance. 

<a id="arucomarkers"></a>
### 1.a ArUco Markers
We chose aruco markers for detecting the corners of the arena (the environment containing the robot, obstacles and goal) and the thymio itself, because they are robust to varying lighting conditions and provide precise detection. We used the original aruco dictionnary as it proved to be more robust to us after trying multiple dictionaries. 
We chose to use the OpenCV aruco detector module for a more straightforward, efficient and robust approach. The steps of the detection can be found in [the documentation](https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html) and [here](https://mmla.gse.harvard.edu/tools/js-aruco-marker-detector/). To get the markers, we used the [ArUco markers generator!](https://chev.me/arucogen/) by Oleg Kalachev. We chose the markers to have a size of $70mm$ and ids: $[0,1,2,10]$ for the arena and $9$ for the Thymio.


<a id="cornerdetection"></a>
#### 1.a.i Corner detection and perspective
`find_aruco_corners_size()`
After getting the aruco markers in the corners of the arena we want to get a conversion factor from pixels to mm to later convert the Thymio's position in pixels to mm for the Kalman filter and controller. We simply take the mean of all 16 sides of the markers and divide it by the real length in mm:
```python
size_aruco_px=[]
for i in range(4):
    side_lengths = [
        np.linalg.norm(corners[i][0,0,:] - corners[i][0,1,:]),  # Top side
        np.linalg.norm(corners[i][0,1,:] - corners[i][0,2,:]),  # Right side
        np.linalg.norm(corners[i][0,2,:] - corners[i][0,3,:]),  # Bottom side
        np.linalg.norm(corners[i][0,3,:] - corners[i][0,0,:])   # Left side
    ]
    size_aruco_px.append(np.mean(side_lengths))
return np.array(outer_corners), np.mean(size_aruco_px)
```
Having the corners of the A0 sheet, we find the maximum width and height of the area defined by the 4 corners, and then compute the perspective transformation matrix $M$, that maps from the distorted space to the new perspective corrected space. Finally, we apply this transformation by using `cv2.warpPerspective()` to isolate our arena, which allows better visualization and detection. This is necessary, as our camera is not exactly above the A0 sheet and perfectly centered ("trapezoidal" deformation), and it makes the edges of the A0 sheet (x and y axis of our coordinate system) parallel to the edges of the corrected image frame (pixel coordinate system), as well as to find the conversion ratio from mm to pixel, needed later for Kalman:
```python
corners, size_aruco_px = find_aruco_corners_size(image)
ordered_corners = order_points(corners)
M = cv2.getPerspectiveTransform(ordered_corners, destination_corners)
max_width_perspective, max_height_perspective = compute_destination_size(ordered_corners)
persp_image = cv2.warpPerspective(image, M, (max_width_perspective, max_height_perspective), flags=cv2.INTER_LINEAR)
pixbymm = size_aruco_px / corner_aruco_size_mm
```

Note that the computation of $M$ is only done once, at the initialization.

<img src="./plots/Imageaftercorrection.png" alt="Arena" width="600">


<a id="thymiodetection"></a>
#### 1.a.ii Thymio detection
`Thymio_position_aruco()`
Similarly, the Thymio's position is found with OpenCV's aruco marker detector, marker id = 9. By doing the mean of the marker's corners, we find $(x,y)$.
```python
idx = np.where(ids == self.Thymio_ID)[0][0] #Thymio's aruco ID is 9
aruco_corners=np.array(corners[idx][0,:,:])
Thymio_x, Thymio_y=aruco_corners.mean(axis=0)
```
To find the angle $\theta$ we take the mean of the angles of the top and bottom edge of the marker with reference to the x-axis:
```python
top_edge=aruco_corners[1,:]-aruco_corners[0,:]
bottom_edge=aruco_corners[2,:]-aruco_corners[3,:]
angle = np.mean([np.arctan2(bottom_edge[1], bottom_edge[0]), 
                np.arctan2(top_edge[1], top_edge[0])])
```



<a id="colordetection"></a>
#### 1.b Color detection
The detection of the obstacle and goal is done in multiple steps in `full_detection_cnt_centroid()`:
1. We create a binary mask with color thresholding. We pick the color thresholds by using an interactive thresholder to find $[{(B_{low}, G_{low}, R_{low})_{green}, (B_{high}, G_{high}, R_{high})_{green}}]$ and $[{(B_{low}, G_{low}, R_{low})_{red}, (B_{high}, G_{high}, R_{high})_{red}}]$. Because we only detect 2 colors: green for the goal and red for the global obstacles, that are fairly distinct we could relax our threshold to have a robust system.
2. For the goal, we take the biggest contour that is in the goal thresholded mask, as there is only 1 green object. Smaller contours are noise.
```python
goal_mask = cv2.inRange(image, THRESHOLD_GOAL_LOW, THRESHOLD_GOAL_HIGH])
goal_contours, _ = cv2.findContours(goal_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
goal_contour = max(goal_contours, key=cv2.contourArea)
```
3. Once the goal is isolated, we find its center, a necessary step for the path planning algorithm.
```python
M = cv2.moments((goal_mask * 255).astype(np.uint8))
Goal_x = int(M["m10"] / M["m00"])
Goal_y = int(M["m01"] / M["m00"])
Goal_center = np.array([Goal_x, Goal_y]).reshape(2, 1)
```
5. For the global obstacles, similarly based on color thresholds, we create a mask with all the global obstacles. Then we remove noise by using the `filter_small_blobs(obstacles_mask, blob_min_size)`, this function will remove connected components with a size smaller than `blob_min_size`, to filter out noise as our global obstacles have significant size. After, `fill_holes` finds the contour for each obstacle with `cv2.findContours()` and then fills them with `cv2.drawContours(thickness=cv2.FILLED)`, to have smooth and well defined obstacles.
Raw mask:
<img src="./plots/goal mask0.png" alt="Arena" width="600">
```python
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.drawContours(filled_mask, contours, -1, 255, thickness=cv2.FILLED)
```
Nice contours:
<img src="./plots/goal mask.png" alt="Arena" width="600">
6. Finally we expand the obstacles with Thymio's radius and mm to pixel conversion ratio, by taking all pixels that are at a certain distance from the obstacle, because we use the L2 norm as a heuristic in the path planning we use the same distance here This is a crucial step for path planning to avoid Thymio and global obstacles collisions.
Here is the distance map for the extended obstacle calculation:
<img src="./plots/distance image.png" alt="Arena" width="600">




Here is the final image for the grid and plots:
<img src="./plots/thresholded image.png" alt="Arena" width="600">
```python
radius = 1 * Thymio_radius_mm * self.pixbymm
distance = cv2.distanceTransform(~obstacle_mask, cv2.DIST_L2, 5)
expanded_obstacle_mask = (distance < radius) * 255
```

At the end we plot the path(red), the measured position(pink), the estimated position (purple) and the keypoints:
<img src="./plots/history.png" alt="Arena" width="600">

This is during the run where we plot the variance (purple for position, pink for angle) and the arrow is the estimated position and angle of the thymio:
<img src="./plots/thymiopicture.png" alt="Arena" width="600">

<a id="kalmanfilter"></a>
## 2. Kalman Filter

<a id="motionmodel"></a>
### 2.a Motion Model
We model the robot's state as $\mathbf{x} = [x, y, \theta]^T$, where:
- $x$ and $y$ are the position coordinates in millimeters.
- $\theta$ is the orientation angle in radians.

The motion of the robot is governed by the kinematic equations for a differential drive robot:
$\begin{pmatrix}
x(t+\Delta t) =x(t) + v \cdot \cos( \theta + \frac{\omega \cdot \Delta t}{2}) \Delta t\\
y(t+\Delta t) = y(t) + v \cdot \sin(\theta + \frac{\omega \cdot \Delta t}{2}) \Delta t\\ 
\theta(t+\Delta t) = \theta(t) + \omega \cdot \Delta t
\end{pmatrix}$

Where the inputs are the left and right motor speeds v_R and v_L that appear through $v = \frac{v_L + v_R}{2}$ and $\omega = \frac{v_L - v_R}{L}$


We do the Runge-Kutta 2 approximation for better accuracy for a still simple and fast model as in this [EKF lecture Princeton](https://www.cs.princeton.edu/courses/archive/fall11/cos495/COS495-Lecture17-EKFLocalization.pdf)



<a id="kalmanequations"></a>
### 2.b Kalman equations
The Kalman filter equations used are the ones of the [lecture on uncertainty](https://moodle.epfl.ch/pluginfile.php/2715133/mod_resource/content/14/Slides%2007_08%20-%20Uncertainties.pdf). We have to use an extended Kalman filter because of the non-linear dynamics. For this we need to calculate the Jacobians of the motion model.
The equations are:
1. Prediction:
$x_t = g(u_t, x_{t-1})$ and
$\Sigma_t = G_t \Sigma_{t-1} G_t^\top + Q_t$
2. Update:
$K_t = \Sigma_t H_t^\top \left( H_t \Sigma_t H_t^\top + R_t \right)^{-1}$, $x_t = x_t + K_t \left( z_t - h(x_t) \right)$ and 
$\Sigma_t = \left( I - K_t H_t \right) \Sigma_t$

Where G and H are the Jacobians of g and h w.r.t x respectively.

However these equations asume Gaussian white noise on the process. However our process noise depends on the input. Indeed because there is no lateral movement if the robot goes in the x direction it's variance cannot be the same as in y. This is already partially taken into account in the equations with $G_t \Sigma_{t-1} G_t^\top$ but then it is hard to find Q if not by fine-tuning. We decided to calculate Q based on Q_control the covariance matrix of the control inputs as in [EKF lecture Princeton](https://www.cs.princeton.edu/courses/archive/fall11/cos495/COS495-Lecture17-EKFLocalization.pdf) or [this paper](https://www.sba.org.br/Proceedings/SBAI/SBAI2017/SBAI17/papers/paper_426.pdf):

We have $Q=G{u,t} \cdot Q_{control} \cdot G{u,t}^\top$ where $G{u,t}$ is the Jacobian of the motion model with respect to the inputs calculated as follows:
$$G_{u,t} =\begin{pmatrix}
\frac{\partial x}{\partial v_L} & \frac{\partial x}{\partial v_R} \\
\frac{\partial y}{\partial v_L} & \frac{\partial y}{\partial v_R} \\
\frac{\partial \theta}{\partial v_L} & \frac{\partial \theta}{\partial v_R}
\end{pmatrix}$$ 
giving
$$G_{u,t} =
\begin{pmatrix}
\frac{\Delta t}{2} \cos(\theta_{\text{mid}}) - \frac{(v_L + v_R) \sin(\theta_{\text{mid}}) \Delta t^2}{4L} & \frac{\Delta t}{2} \cos(\theta_{\text{mid}}) + \frac{(v_L + v_R) \sin(\theta_{\text{mid}}) \Delta t^2}{4L} \\
\frac{\Delta t}{2} \sin(\theta_{\text{mid}}) + \frac{(v_L + v_R) \cos(\theta_{\text{mid}}) \Delta t^2}{4L} & \frac{\Delta t}{2} \sin(\theta_{\text{mid}}) - \frac{(v_L + v_R) \cos(\theta_{\text{mid}}) \Delta t^2}{4L} \\
\frac{\Delta t}{L} & -\frac{\Delta t}{L}
\end{pmatrix}$$

where $\theta_{\text{mid}}=\theta + \frac{\omega \cdot \Delta t}{2}$ from RK2 as above.



<a id="covariancemeasurements"></a>
### 2.c Covariance measurements
The covariances are key components in the Kalman Filter. To determine the different covariances matrices: Q of the process and R of the measurements we did the following:



<a id="processcovarianceq"></a>
#### 2.c.i Process Covariance Q
The process covariance represents the variance of the error due to the model. Errors can appear either because of our simplified model or because of wheel slippage etc. We had two different approaches to measure Q:

- First, as discussed before we directly used $Q$ in the predict step without the Jacobian $G_u$. For this we runned for different motor speeds and delta t's the Thymio and compared the predicted position and the measured with a precise ruler and a protractor.By assuming the covariance matrix was diagonal and $var(x)=var(y)$ we then computed the variance in $\frac{mm}{s}^2$ with `np.var((y_meas-y_pred)/dt)`. In the code we multiplied by $dt^2$ to get the covariance in $mm^2$.

- Secondly we tried with the method that seemed more correct and which is more widely supported by lecture notes and papers by taking into account the impact of the input on the motion through the Jacobian $G_u$. We then needed to find $Q_{control}$ because $Q=G{u,t} \cdot Q_{control} \cdot G{u,t}^\top$. We supposed $Q_{control}$ is diagonal and white noise because noise on speed measurement left and right are supposedly independent, gaussian. So we let the robot run for an amount of time dt at a certain target speed and we did the variance `np.var(v_L_meas-v_L_meas_target)` and the covariance becomes `Q_ctrl=np.diag([var(v_L_meas-v_L_meas_target),var(v_L_meas-v_R_meas_target)])` which is in Thymio speed (pwm) and so we multiply by the speed factor squared to get $\frac{mm}{s}^2$ and use it in our equations. We measured `self.kalman_Q_ctrl=np.diag([31, 29])`


- The second method worked perfectly and we did no have to tune (not even a little bit) the variances. When the camera is obscured the variance increases as an ellipse in the direction of motion. This is expected because the robot cannot do lateral movements so the variances laterally increases little. We plotted the ellipse as representing the 95% confidence zone ($2\cdot sigma$) for the position and the arc as the 95% confidence zone for the angle.
<img src="./plots/Kalman.png" alt="Kalman" width="600">




<a id="measurementscovariancer"></a>
#### 2.c.ii Measurements Covariance $R$
The covariance $R$ accounts for the imprecision of the sensors that measures the states. In our case the only sensor we have for the states (H matrix) is the camera. $R$ is found by keeping the system's output constant and measuring the variance around the mean of the measurements. In our case we keep the robot immobile and measure the position using the camera and calculate the variance. By supposing that the variances are independent $R$ is diagonnal and simply `R=np.diag([var_x,var_y,var_z])` multiplied by the scaling factor to go from $pix^2$ to $mm^2$ for the first two variances. We found $10^{-7}$ because the aruco detection and the camera measurement is very precise but the precision is in pixel so the variance is at least 1 $pix^2$ which gives 4 mm^2 (what we used). We put 2 degrees for the angle to not have 0. Which is what we choose in the end. This method was found in "Probabilistic Robotics" by Sebastian Thrun, Wolfram Burgard, and Dieter Fox 


<a id="codeimplementation"></a>
### 2.d Code implementation
In the code the prediction is done in the thymio class through `kalman_predict_state()`. It updates the estimated position (`xytheta_est`) to the predicted one. The update step is done with `kalman_update_state()`. It updates the estimated position (`xytheta_est`) to a fusion of the measurements and the predicted position.

If no Thymio is found, the update step is not done as there are no measurements in position but only in speed (the inputs)


<a id="kidnapping"></a>
## 3. Kidnapping

At every step, the thymio checks if it has been kidnapped based on the ```prox.ground.delta``` value. If they are too low, that means the robot is no longer on the white sheet, so we stop set the motors to 0 to stop moving while off the ground. Once the robot is back on the ground, we update the estimated position multiple time so the position given by the kalman filter has time to re-align itself with the more trusted position given by the camera:

```python
#snippet of code from the main :
path_planning = True
for i in range(30):
    cam.get_image()
    cam.correct_perspective_aruco(get_matrix = False)
    Thymio.Thymio_position_aruco(cam.persp_image)
    Thymio.delta_time_update()
    v_L, v_R = await gather_data(node)
    Thymio.kalman_predict_state(v_L, v_R)  
    if Thymio.Thymio_detected:  #only update if Thymio detected
        Thymio.kalman_update_state()
```

Once the position is up to date and because we put ```path_planning``` to **True**, a new path according to the new position will be calculated during the next step and the thymio can go back to moving towards his goal.
<img src="./plots/kidnaphistory.png" alt="Arena" width="600">

<a id="pathplanning"></a>
## 4. Path Planning

<a id="astar"></a>
### 4.a A*
We choose the A star algorithm for path planning because:
- it is complete and will always give us a solution
- an image is like a grid so the preprocessing to get the grid is straightforward
- we can choose it's precision/running time by tuning the grid size: it is only done at the beginning (where we don't care about time) and at after each local avoidance so we can have a precise planning at the beginning and a faster after local avoidance
The downsides are:
- We choose to consider 8 neighbors and the L2 norm as heuristic because the robot can go diagonnaly. However, the robot is not limited to 45 degrees angles so the soltuion is not optimal for the robot
- we need to extract keypoints to give more latitude to the controller to o to each of them and to avoid too many changes:if the optimal is to go 30 degrees down right the path will be a lot of variations between going straight/down/45 degrees. With key points the robot only need to go from the beginning and end of the slope and could do 30 degrees directly optimizing a bit.

The implemenation of the A* is quite similar to the exercice session except a $\sqrt{2}$ to account for a longer distance when doing a diagonal move:
```python
if (neighbor[0]==current_pos[0]) or (neighbor[1]==current_pos[1]): #if goes straight
    tentative_g_cost = g_costs[current_pos]+(map_grid[neighbor]) #cost is 1 y default on the map_grid
else:
    tentative_g_cost = g_costs[current_pos]+(map_grid[neighbor])*np.sqrt(2) #going diagonnaly is going further

```

A plot of the a star with the prints of the code:
<img src="./plots/Astarandprints.png" alt="Arena" width="600">


<a id="keypoints"></a>
### 4.b Finding keypoints of the path

```python
#snippet of code from the function find_keypoints(path) : 

# direction vectors
dir_previous = (current[0] - previous[0], current[1] - previous[1])
dir_next = (next[0] - current[0], next[1] - current[1])

if (abs(find_rotation(dir_previous, dir_next)) > ANGLE_THRESHOLD): # significant change of direction
    keypoints.append(current)
    counter = 1
elif (counter >= COUNTER_THRESHOLD):  # ensures there isn't too much space between keypoints 
    keypoints.append(current)
    counter = 1
else:
    counter += 1
```

Given the size of the Thymio compared to the size of the pixels, we won't follow the path pixel by pixel but instead extract keypoints that will serve as intermediate goals for the Thymio.
For cells every ```STEP``` cells, we study the vectors between the previous cell (```STEP``` cells before) and the current cell, and between the current cell and the next cell (```STEP``` cells after).
In the function ```find_rotation(dir_previous, dir_next)``` we calculate how much the robot has to turn to change from ```dir_previous``` to ```dir_next```, and if this angle surpasses a threshold (meaning the turn is non negligable), we add the current cell to the list of keypoints of the path. Even if the angles are found insignificant, a cell is added to the keypoints every few steps to make sure we don't ignore too many small changes of direction that could become significant when added together.

In this algorithm we chose to study the path in steps of size ```STEP``` instead of considering every single one of the cells because the cells of the grid are very small compared to the Thymio. 

```python
def find_rotation(dir_previous, dir_next):
    det = dir_previous[0] * dir_next[1] - dir_previous[1] * dir_next[0]
    dot_product = dir_previous[0] * dir_next[0] + dir_previous[1] * dir_next[1]
    theta = np.arctan2(det, dot_product)  # angle between the two directions [rad]
    return theta
```
We compute the determinant and the dot product of the two vectors to use ```math.atan2(det, dot_product)```, which allows us to find the signed angle between the two vectors. atan2() can find angles over the whole trigonometric circle (using the information given by the sign of the dot_product), not limiting itself to only two quadrants like atan().

**Example : Calling ```find_keypoints(path)``` on a random path :**

```python
#parameters :
ANGLE_THRESHOLD = math.radians(20)
STEP = 2
COUNTER_THRESHOLD = 5 #max number of steps between keypoints
```

<img src="./images/path_example.jpg" alt="Diagram" width="200">


We created and displayed a grid with a random path to test the function `find_keypoints(path)`.

Here, we can see that most of the keypoints are added because of direction changes, except the fifth keypoint (in the middle of the third diagonal) that was added because of `COUNTER_THRESHOLD`. Indeed it is on the 10th cell after the previous keypoints (`COUNTER_THRESHOLD`*`STEP`=10).

<a id="motioncontrol"></a>
## 5. Motion control

<a id="motorspeeds"></a>
### 5.a Setting the motors' speed

`limit_speed(v)` : limits the speed to 500, the maximum input accepted by the thymio

`set_motors(node, v_ml, v_mr)` : makes sure the speeds demanded do not exceed the maximum speed of 500, transforms the wanted speeds to int type (compatible with thymio), and then applies the adapted speeds to the motors.



<a id="controller"></a>
### 5.b Controller

Diagram showing the thymio, the goal, and the variables used for the controller :

<img src="./images/schema_astolfi.jpg" alt="Diagram" width="200">

We chose to design a controller because it makes the Thymio moves more smoothly than advancing segment by segment and making the robot turn on itself between each segment. Indeed it makes a mix of both the wanted translational and rotational velocities to set the motors' speed.

At first we implemented an Astolfi controller, but it calculates the translational speed proportionnaly to the distance to the goal, and therefore slows down as it gets closer to the goal. We wanted the thymio to have a constant speed when following his path, and therefore we made some changes to the controller :

We start from a baseline translational velocity `SPEED`, which for each wheel we will adapt depending on delta_angle, to obtain a curved and smooth trajectory.

```python
v = SPEED                                                  #translational velocity PWM
omega = k_alpha*(delta_angle) - k_beta*(delta_angle+theta) #rotational velocity [rad/s]

#Calculate motor speed
v_ml = (v+omega*L_AXIS) #PWM
v_mr = (v-omega*L_AXIS) #PWM
```

We tuned the controllers parameters by testing.

* **`k_alpha`** : controls how much the robot turns to align itself with the goal. Depends on `SPEED` : The higher the basic speed is, the more reactive the thymio needs to be to changes of directions.
    * **`k_alpha` too low :** the thymio won't change direction fast enough, it will react too slow to follow the path (therefore not following well the shortest path and risking going over global obtacles)
     * **`k_alpha` too high** : the robot will turn to aggressively and overshoot the turns
and k_beta is a damping term (to stabilize the robot's orientation when reaching the goal).

* **`k_beta`** : damping term if the robot oscillates around his wanted direction. The thymio didn't oscillate and adding a `k_beta` different than zero only made it react too slowly (too much damping), so we decided `k_beta` wasn't needed in this context. We left it in the code because it may be useful to add a damping term in other situations, if the robot's speed was way higher for example, so the code can be easily adapted to other contexts.



<a id="localavoidance"></a>
### 5.c Local Avoidance

At each iterations, we call `check_obstacle(prox_values)` to check if the thymio is confronted to a local obstacle. If so the robot will keep applying Braitenberg's logic while it can see the obstacle.

`check_obstacle(prox_values)` : check if there is an obstacle based on the maximum proximity sensor value

`avoid_obstacle(prox_values)` : apply braitenberg's logic, the motors' speed is proportional to the proximity sensors values

```python
braitenberg = [-2/300, -10/300, 25/300, 11/300, 3/300] #left to right
v_mr, v_ml = SPEED, SPEED
for i in range (len(prox_values)) :
    v_ml -= braitenberg[i] * prox_values[i]
    v_mr += braitenberg[i] * prox_values[i]
```
We tuned braitenberg's parameters by testing. The values linked to the proximity sensors closer to the center are higher than those linked to the sensors near the edge of the robot because the thymio needs to turn faster to avoid an obstacle directly in front of him than an obstacle more shifted to his left or right.
The center value is positive like the values linked to the sensors on the right, therefore when faced with an obstacle in the middle the robot will choose to turn to the left. The other values are not completely symetrical so that the robot will turn to the left instead of not turning at all when faced with obstacles symetrical on his sides.
Here we can see the replan:
<img src="./plots/historyobstacle.png" alt="Arena" width="600">


<a id="annexe"></a>
## Annexe

Initially, at the start of the project, we chose to work with a black background to easily identify the white Thymio using computer vision algorithms.
To isolate the arena we used a canny edge filter `cv2.Canny` (Gaussian blur $\rightarrow$ Sobel $\rightarrow$ non maxima suppresion $\rightarrow$ hysteresis), to identify the arena borders and then `cv2.approxPolyDP()` to approximate the borders to a rectangular shape and get its corners. With these corners we did perspective correction with `cv2.getPerspectiveTransform` and `cv2.warpPerspective` same as the current approach. 

<img src="https://i.ibb.co/WBKxfmh/image-2024-12-05-203548559.png" alt="Canny1" width="600">

After some color thresholding, we were able to get a mask of the Thymio, which gives the centroid coordinates. However, there was still the challenge of finding the Thymio's orientation. A solution was to use the Hough Transform `cv2.HoughLinesP()` to remove from the Thymio borders, the 3 straights borders, to only keep the curved border, also called the *nose* of the robot. With the curved border, we found the two edge points, computed the center ("center of the nose") and drew a line from the robot's center to the curved border center. The intersection of that line with the curved border gave the *nose point*. With the *nose point* and robot center we can compute the robot's orientation.

However this approach, is tune heavy. It required to finely tune first, a canny edge filter and $\epsilon$ for the polynomial approximation. Then the color thresholds to separate the robot, goal and obstacles from the background. A finally second canny filter before running the hough transform.

Excluding the colors, this required to tune 10 parameters (3 per canny, 3 hough and polynomial approximation). This approach was cumbersome and not robust to intensity changes. That is why we decided to switch to arucos for the arena borders and to place an aruco on the Thymio for fast and robust identification, as arucos give position and orientation! We switched to a white background to easily detect them.

<img src="https://i.ibb.co/SQsvjK4/image-2024-12-05-203955707.png" alt="Canny2" width="600">

# PROJECT :

There is a notebook main and a .py to run the code. Here is a nice view of a copy:

```python
%reload_ext autoreload
%autoreload 2

import numpy as np
import cv2
from tdmclient import ClientAsync, aw
from utils.camera import *
from utils.thymio import *
from utils.path import *
from utils.motion_control import *
from utils.buttons import*
from utils.drawings import *
from utils.color_thresholds import load_thresholds
###########################################################
# Parameters
###########################################################
COLOR_OBSTACLE = np.array([[80, 40, 0, 255, 80, 20]]) # BGR
COLOR_GOAL = np.array([0, 77, 0, 68, 255, 118])       # BGR
#COLOR_OBSTACLE = load_thresholds("color_obstacles.txt").reshape(1,-1) # run the notebook inside utils and save thresholds
#COLOR_GOAL = load_thresholds("color_goal.txt")
###########################################################
# Main Code
###########################################################
from tdmclient import ClientAsync, aw
client = ClientAsync()

async def main():

    node = await client.wait_for_node()
    aw(node.lock())
    
    print("Press forward button to start the program")
    beginning = False
    while(beginning == False) :
        beginning = await wait_for_start_button(node, client)
        time.sleep(0.3)
    cv2.destroyAllWindows()
    print("Starting the program")

    program = '''
        # Turn off LEDs during initialization
        call leds.temperature(0, 0)
        call leds.prox.v(0, 0)
        '''
    await node.compile(program)
    await node.run()
    
    # Camera initialization
    cam = camera_class(COLOR_OBSTACLE,COLOR_GOAL)
    while not cam.corners_found:
        cam = camera_class(COLOR_OBSTACLE, COLOR_GOAL)

    # Thymio initialization
    Thymio = Thymio_class(cam)

    path_planning = True
    local_avoidance = False
    step = 0
    kidnapped = False
    path_img = None 
    path_img_hist = []

    while True :  
        
        step = step + 1
        
        # Update Image
        cam.get_image()
        cam.correct_perspective_aruco(get_matrix = False)
        
        # Thymio Position and motor
        if not kidnapped: 
            Thymio.Thymio_position_aruco(cam.persp_image)
        Thymio.delta_time_update()

        #Kalman Filter
        v_L, v_R = await gather_data(node)
        Thymio.kalman_predict_state(v_L, v_R)  
        if Thymio.Thymio_detected:  # Only update if Thymio detected
            Thymio.kalman_update_state()

        #Update history for final plot
        Thymio.xytheta_meas_hist = np.vstack((Thymio.xytheta_meas_hist, Thymio.xytheta_meas))
        Thymio.xytheta_est_hist = np.vstack((Thymio.xytheta_est_hist, Thymio.xytheta_est))
        
        #Check kidnapping
        if(await check_kidnapping(node)):
            draw_on_image(cam, Thymio, path_img)
            if(kidnapped == False):
                print("Thymio was kidnapped !")
            kidnapped = True
            Thymio.Thymio_detected=False
            await set_motors(node, 0, 0)
            continue
        if(kidnapped):
            kidnapped = False
            path_planning = True
            #do_plot = True
            for i in range(30):
                cam.get_image()
                cam.correct_perspective_aruco(get_matrix = False)
                Thymio.Thymio_position_aruco(cam.persp_image)
                Thymio.delta_time_update()
                v_L, v_R = await gather_data(node)
                Thymio.kalman_predict_state(v_L, v_R)  
                if Thymio.Thymio_detected:  # Only update if Thymio detected
                    Thymio.kalman_update_state()
                draw_on_image(cam, Thymio, path_img)
            print("Thymio back on the ground")
            continue
            
        #Path Planning
        if path_planning:
            if Thymio.target_keypoint is None or not np.any(Thymio.target_keypoint):
                do_plot = True
            grid = discretize_image_eff(cam.thresholded_image)
            # Careful! Image frame's first coord (x) is pointing right but in a matrix the first coordinate (rows) is pointing down so they must be inverted
            found, path, _, _ = a_star_search(
                grid,
                grid1_coord2grid2_coord(
                    np.array([Thymio.xytheta_est[1], Thymio.xytheta_est[0]]),
                    cam.persp_image,
                    grid
                ),
                grid1_coord2grid2_coord(
                    np.array([cam.goal_center[1], cam.goal_center[0]]),
                    cam.persp_image,
                    grid
                ),
                do_plot
            )

            if not found:
                print("Couldn't find path, stopping the mission")
                draw_on_image(cam, Thymio, None)
                cv2.waitKey(10)
                aw(node.stop())
                aw(node.unlock())
                break

            # Convert path coordinates for plotting
            path_img = grid1_coord2grid2_coord(path, grid, cam.persp_image)
            path_img = path_img[::-1]

            keypoints = find_keypoints(path_img)
            path_img_hist.append(path_img)
            Thymio.keypoints=keypoints[1:]
            Thymio.target_keypoint = Thymio.keypoints[0]
            
            do_plot = False
            path_planning = False
        
        #Obstacle detection
        prox_values = await get_prox(node, client)
        if (check_obstacle(prox_values)):
            if not local_avoidance:
                print("Obstacle!")
           
            local_avoidance = True
            v_ml, v_mr = avoid_obstacle(prox_values)
            draw_on_image(cam, Thymio, path_img)
            await set_motors(node, v_ml, v_mr)
            continue
        else:
            if local_avoidance:
                print("Recalculating path")
                await set_motors(node, 1.3*SPEED, 1.3*SPEED) # Move forward to leave the obstacle behind while recalculating path
                for i in range(20):
                    cam.get_image()
                    cam.correct_perspective_aruco(get_matrix = False)
                    Thymio.Thymio_position_aruco(cam.persp_image)
                    Thymio.delta_time_update()
                    v_L, v_R = await gather_data(node)
                    Thymio.kalman_predict_state(v_L, v_R)  
                    if Thymio.Thymio_detected:  # Only update if Thymio detected
                        Thymio.kalman_update_state()
                    draw_on_image(cam, Thymio, path_img)
                    Thymio.xytheta_meas_hist = np.vstack((Thymio.xytheta_meas_hist, Thymio.xytheta_meas))
                    Thymio.xytheta_est_hist = np.vstack((Thymio.xytheta_est_hist, Thymio.xytheta_est))
                    time.sleep(0.0175)
                path_planning = True
                local_avoidance = False
                continue
        
        #Motion control    
            else:
                if((step % 5) == 0):
                    if(Thymio.reached_goal()):
                        if(len(Thymio.keypoints) <= 1): #Thymio found the goal
                            print("Mission accomplished") 
                            aw(node.stop())
                            aw(node.unlock())
                            draw_history(cam, Thymio, path_img_hist, keypoints)
                            break
                        Thymio.keypoints = Thymio.keypoints[1:]
                        Thymio.target_keypoint = Thymio.keypoints[0]
                    v_ml, v_mr = motion_control(Thymio)
                    await set_motors(node, v_ml, v_mr)
                draw_on_image(cam, Thymio, path_img)

        #Check stop            
        if(await check_stop_button(node, client)):
            aw(node.stop())
            aw(node.unlock())
            draw_history(cam, Thymio, path_img_hist, keypoints)
            break
    
    cam.cam.release()

# Run the main asynchronous function
while True:
    client.run_async_program(main)
```